In [1]:
%load_ext autoreload
%autoreload 2

Import varie e inizializzazione delle variabili dei modelli e del tuner

In [2]:
import os
from tensorflow import keras
from sklearn.model_selection import train_test_split
from IRESNs_tensorflow.time_series_datasets import *
from IRESNs_tensorflow.models import ESN, IRESN, IIRESN

PROJECT_ROOT = os.path.abspath(os.getcwd())
DATASET_NAME = "CharacterTrajectories"
DATASET_DIR = os.path.join(PROJECT_ROOT, "datasets")

READOUT_ACTIVATION_BINARY = keras.activations.sigmoid
LOSS_FUNCTION_BINARY = keras.losses.BinaryCrossentropy()

READOUT_ACTIVATION = keras.activations.softmax
LOSS_FUNCTION = keras.losses.SparseCategoricalCrossentropy()

# Dataset dependent settings
RESERVOIRS = 3
OUTPUT_UNITS = 20

# Tuner settings
MAX_EPOCHS = 3
PATIENCE = 1
MAX_TRIALS = 2
OVERWRITE = True
TRIALS = 1

BENCHMARKS_TRIALS = 1

MINVAL = 0.01
MAXVAL = 1.5

/dati/luca/Uni-Luca/Tesi/progetto/venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/dati/luca/Uni-Luca/Tesi/progetto/venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/dati/luca/Uni-Luca/Tesi/progetto/venv/lib/python3.9/site-packages/sktime/datatypes/_series/_check.py:42: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  VALID_INDEX_TYPES = (pd.Int64Index, pd.RangeIndex, pd.PeriodIndex, pd.DatetimeIndex

Definizione della funzione utilizzata dal tuner per istanziare i modelli per la model selection.
Nella seguente cella si definisce un modello di tipo ESN.

In [3]:
def build_ESN(tuner):
    ESN_model = ESN(units=100,
                    connectivity=tuner.Float('connectivity', min_value=0.01, max_value=1.),
                    spectral_radius=tuner.Float('spectral radius', min_value=MINVAL, max_value=MAXVAL),
                    input_scaling=tuner.Float('input scaling', min_value=MINVAL, max_value=MAXVAL),
                    bias_scaling=tuner.Float('bias scaling', min_value=MINVAL, max_value=MAXVAL),
                    leaky=tuner.Float('leaky', min_value=0.01, max_value=1.0),
                    output_units=OUTPUT_UNITS,
                    readout_activation=READOUT_ACTIVATION,
                    )

    alpha = tuner.Float('learning rate', min_value=1e-5, max_value=1e-1, sampling='log')
    ESN_model.compile(
        optimizer=keras.optimizers.Adam(alpha),
        loss=LOSS_FUNCTION,
        metrics=['accuracy'],
    )

    return ESN_model

In seguito vengono definiti due tipi di modelli
- IRESN ha #'RESERVOIRS' al suo interno di tipo ESN tutti della stessa dimensione non comunicanti.
- IIRESN #'RESERVOIRS' al suo interno di tipo ESN tutti della stessa dimensione comunicanti.

In [4]:
def build_IRESN(tuner):
    connectivity = [tuner.Float('connectivity ' + str(i), min_value=0.01, max_value=1.) for i in range(RESERVOIRS)]
    normalization = [tuner.Float('spectral radius ' + str(i), min_value=MINVAL, max_value=MAXVAL) for i in
                     range(RESERVOIRS)]

    IRESN_model = IRESN(sub_reservoirs=RESERVOIRS,
                        output_units=OUTPUT_UNITS,
                        units=100,
                        connectivity=connectivity,
                        normalization=normalization,
                        input_scaling=tuner.Float('input scaling', min_value=MINVAL, max_value=MAXVAL),
                        bias_scaling=tuner.Float('bias_scaling', min_value=MINVAL, max_value=MAXVAL),
                        leaky=tuner.Float('leaky', min_value=0.01, max_value=1.0),
                        readout_activation=READOUT_ACTIVATION,
                        )

    alpha = tuner.Float('learning rate', min_value=1e-5, max_value=1e-1, sampling='log')
    IRESN_model.compile(
        optimizer=keras.optimizers.Adam(alpha),  # keras.optimizers.RMSprop(alpha),
        loss=LOSS_FUNCTION,
        metrics=['accuracy'],
    )
    return IRESN_model


def build_IIRESN(tuner):
    norm = tuner.Float('norm', min_value=MINVAL, max_value=MAXVAL)
    normalization = [[tuner.Float('spectral radius ' + str(i), min_value=MINVAL, max_value=MAXVAL) if i == j else
                      norm
                      for i in range(RESERVOIRS)]
                     for j in range(RESERVOIRS)]

    ic_density = tuner.Float('connectivity X->Y', min_value=0., max_value=1.)
    connectivity = [[tuner.Float('connectivity ' + str(i), min_value=0., max_value=1.) if i == j else
                     ic_density
                     for i in range(RESERVOIRS)]
                    for j in range(RESERVOIRS)]

    IIRENS_model = IIRESN(units=100,
                          sub_reservoirs=RESERVOIRS,
                          connectivity=connectivity,
                          normalization=normalization,
                          input_scaling=tuner.Float('input scaling', min_value=MINVAL, max_value=MAXVAL),
                          bias_scaling=tuner.Float('bias_scaling', min_value=MINVAL, max_value=MAXVAL),
                          leaky=tuner.Float('leaky', min_value=0.01, max_value=1.0),
                          output_units=OUTPUT_UNITS,
                          readout_activation=READOUT_ACTIVATION,
                          )

    alpha = tuner.Float('learning rate', min_value=1e-5, max_value=1e-1, sampling='log')
    IIRENS_model.compile(
        optimizer=keras.optimizers.Adam(alpha),  # keras.optimizers.RMSprop(alpha),
        loss=LOSS_FUNCTION,
        metrics=['accuracy'],
    )
    return IIRENS_model

In seguito vengono definiti due varianti ai modelli definiti precedentemente
- IRESNvsr sub-reseroir di dimensione variabile, definita tramite il parametro di funzione 'vsr'.
- IIRESNvsr sub-reseroir di dimensione variabile, definita tramite il parametro di funzione 'vsr'.

In [5]:
# Generate and Normalize the partition vector. sum(partitions) == 1.
def generate_partitions(tuner):
    partitions = [tuner.Float('partition ' + str(i), min_value=0.1, max_value=1.0) for i in range(RESERVOIRS)]
    total = sum(partitions)
    partitions = list(map(lambda _x: 0 if total == 0 else _x / total, partitions))
    return partitions


def build_IRESNvsr(tuner):
    sr = [tuner.Float('spectral radius ' + str(i), min_value=MINVAL, max_value=MAXVAL) for i in range(RESERVOIRS)]
    connectivity = [tuner.Float('connectivity ' + str(i), min_value=0., max_value=1.) for i in range(RESERVOIRS)]

    IRESNvsr_model = IRESN(units=100,
                           sub_reservoirs=RESERVOIRS,
                           output_units=OUTPUT_UNITS,
                           connectivity=connectivity,
                           normalization=sr,
                           vsr=generate_partitions(tuner),
                           input_scaling=tuner.Float('input scaling', min_value=MINVAL, max_value=MAXVAL),
                           bias_scaling=tuner.Float('bias_scaling', min_value=MINVAL, max_value=MAXVAL),
                           leaky=tuner.Float('leaky', min_value=0.01, max_value=1.0),
                           readout_activation=READOUT_ACTIVATION,
                           )

    alpha = tuner.Float('learning rate', min_value=1e-5, max_value=1e-1, sampling='log')
    IRESNvsr_model.compile(
        optimizer=keras.optimizers.Adam(alpha),  # keras.optimizers.RMSprop(alpha),
        loss=LOSS_FUNCTION,
        metrics=['accuracy'],
    )
    return IRESNvsr_model


def build_IIRESNvsr(tuner):
    ic_norm = tuner.Float('norm 2', min_value=MINVAL, max_value=MAXVAL)
    normalization = [[tuner.Float('spectral radius ' + str(i), min_value=MINVAL, max_value=MAXVAL) if i == j else
                      ic_norm
                      for i in range(RESERVOIRS)]
                     for j in range(RESERVOIRS)]

    ic_density = tuner.Float('connectivity X->Y', min_value=0., max_value=1.)
    connectivity = [[tuner.Float('connectivity ' + str(i), min_value=0., max_value=1.) if i == j else
                     ic_density
                     for i in range(RESERVOIRS)]
                    for j in range(RESERVOIRS)]

    IIRESNvsr_model = IIRESN(units=100,
                             sub_reservoirs=RESERVOIRS,
                             connectivity=connectivity,
                             normalization=normalization,
                             vsr=generate_partitions(tuner),
                             input_scaling=tuner.Float('input scaling', min_value=MINVAL, max_value=MAXVAL),
                             bias_scaling=tuner.Float('bias_scaling', min_value=MINVAL, max_value=MAXVAL),
                             leaky=tuner.Float('leaky', min_value=0.01, max_value=1.0),
                             output_units=OUTPUT_UNITS,
                             readout_activation=READOUT_ACTIVATION,
                             )

    alpha = tuner.Float('learning rate', min_value=1e-5, max_value=1e-1, sampling='log')
    IIRESNvsr_model.compile(
        optimizer=keras.optimizers.Adam(alpha),  # keras.optimizers.RMSprop(alpha),
        loss=LOSS_FUNCTION,
        metrics=['accuracy'],
    )
    return IIRESNvsr_model

In questi modelli viene tolto il parametro vsr e i valori di normalizzazione dei sub-reservoir ma aggiunto il global spectral radius.

In [6]:
def build_IRESNgsr(tuner):
    connectivity = [tuner.Float('connectivity ' + str(i), min_value=0., max_value=1.) for i in range(RESERVOIRS)]

    IRESNvsr_model = IRESN(units=100,
                           sub_reservoirs=RESERVOIRS,
                           output_units=OUTPUT_UNITS,
                           connectivity=connectivity,
                           gsr=tuner.Float('global spectral radius', min_value=MINVAL, max_value=MAXVAL),
                           input_scaling=tuner.Float('input scaling', min_value=MINVAL, max_value=MAXVAL),
                           bias_scaling=tuner.Float('bias_scaling', min_value=MINVAL, max_value=MAXVAL),
                           leaky=tuner.Float('leaky', min_value=0.01, max_value=1.0),
                           readout_activation=READOUT_ACTIVATION,
                           )

    alpha = tuner.Float('learning rate', min_value=1e-5, max_value=1e-1, sampling='log')
    IRESNvsr_model.compile(
        optimizer=keras.optimizers.Adam(alpha),  # keras.optimizers.RMSprop(alpha),
        loss=LOSS_FUNCTION,
        metrics=['accuracy'],
    )
    return IRESNvsr_model


def build_IIRESNgsr(tuner):
    ic_density = tuner.Float('connectivity X->Y', min_value=0., max_value=1.)
    connectivity = [[tuner.Float('connectivity ' + str(i), min_value=0., max_value=1.) if i == j else
                     ic_density
                     for i in range(RESERVOIRS)]
                    for j in range(RESERVOIRS)]

    IIRESNvsr_model = IIRESN(units=100,
                             sub_reservoirs=RESERVOIRS,
                             connectivity=connectivity,
                             gsr=tuner.Float('global spectral radius', min_value=MINVAL, max_value=MAXVAL),
                             input_scaling=tuner.Float('input scaling', min_value=MINVAL, max_value=MAXVAL),
                             bias_scaling=tuner.Float('bias_scaling', min_value=MINVAL, max_value=MAXVAL),
                             leaky=tuner.Float('leaky', min_value=0.01, max_value=1.0),
                             output_units=OUTPUT_UNITS,
                             readout_activation=READOUT_ACTIVATION,
                             )

    alpha = tuner.Float('learning rate', min_value=1e-5, max_value=1e-1, sampling='log')
    IIRESNvsr_model.compile(
        optimizer=keras.optimizers.Adam(alpha),  # keras.optimizers.RMSprop(alpha),
        loss=LOSS_FUNCTION,
        metrics=['accuracy'],
    )
    return IIRESNvsr_model


Modelli definiti come nella cella precedente ma viene reintrodotta la normalizzazione dei sub-reservoir e rimane il global spectral radius.

In [7]:
def build_IRESN_norm_gsr(tuner):
    connectivity = [tuner.Float('connectivity ' + str(i), min_value=0.01, max_value=1.) for i in range(RESERVOIRS)]
    sr = [tuner.Float('spectral radius ' + str(i), min_value=MINVAL, max_value=MAXVAL) for i in range(RESERVOIRS)]

    IRESN_model = IRESN(units=100,
                        sub_reservoirs=RESERVOIRS,
                        connectivity=connectivity,
                        normalization=sr,
                        gsr=tuner.Float('global spectral radius', min_value=MINVAL, max_value=MAXVAL),
                        input_scaling=tuner.Float('input scaling', min_value=MINVAL, max_value=MAXVAL),
                        bias_scaling=tuner.Float('bias_scaling', min_value=MINVAL, max_value=MAXVAL),
                        leaky=tuner.Float('leaky', min_value=0.01, max_value=1.0),
                        output_units=OUTPUT_UNITS,
                        readout_activation=READOUT_ACTIVATION,
                        )

    alpha = tuner.Float('learning rate', min_value=1e-5, max_value=1e-1, sampling='log')
    IRESN_model.compile(
        optimizer=keras.optimizers.Adam(alpha),  # keras.optimizers.RMSprop(alpha),
        loss=LOSS_FUNCTION,
        metrics=['accuracy'],
    )
    return IRESN_model


def build_IIRESN_norm_gsr(tuner):
    norm = tuner.Float('norm', min_value=MINVAL, max_value=MAXVAL)
    normalization = [[tuner.Float('spectral radius ' + str(i), min_value=MINVAL, max_value=MAXVAL) if i == j else norm
                      # 0 sulla diagonale limit su tutte le posizioni off-diagonali
                      for i in range(RESERVOIRS)]
                     for j in range(RESERVOIRS)]

    ic_density = tuner.Float('connectivity X->Y', min_value=0., max_value=1.)
    connectivity = [[tuner.Float('connectivity ' + str(i), min_value=0., max_value=1.) if i == j else
                     ic_density
                     for i in range(RESERVOIRS)]
                    for j in range(RESERVOIRS)]

    IIRENS_model = IIRESN(units=100,
                          sub_reservoirs=RESERVOIRS,
                          connectivity=connectivity,
                          normalization=normalization,
                          gsr=tuner.Float('global spectral radius', min_value=MINVAL, max_value=MAXVAL),
                          input_scaling=tuner.Float('input scaling', min_value=MINVAL, max_value=MAXVAL),
                          bias_scaling=tuner.Float('bias_scaling', min_value=MINVAL, max_value=MAXVAL),
                          leaky=tuner.Float('leaky', min_value=0.01, max_value=1.0),
                          output_units=OUTPUT_UNITS,
                          readout_activation=READOUT_ACTIVATION,
                          )

    alpha = tuner.Float('learning rate', min_value=1e-5, max_value=1e-1, sampling='log')
    IIRENS_model.compile(
        optimizer=keras.optimizers.Adam(alpha),  # keras.optimizers.RMSprop(alpha),
        loss=LOSS_FUNCTION,
        metrics=['accuracy'],
    )
    return IIRENS_model

In [8]:
train_path = os.path.join(DATASET_DIR, DATASET_NAME, DATASET_NAME + '_TRAIN.ts')
test_path = os.path.join(DATASET_DIR, DATASET_NAME, DATASET_NAME + '_TEST.ts')

x_train_all, y_train_all = load_sktime_dataset(train_path)
x_test, y_test = load_sktime_dataset(test_path)

x_train, x_val, y_train, y_val = train_test_split(x_train_all, y_train_all,
                                                  test_size=0.33, random_state=42, stratify=y_train_all)

train_set = (x_train, y_train)
val_set = (x_val, y_val)
test_set = (x_test, y_test)

/dati/luca/Uni-Luca/Tesi/progetto/venv/lib/python3.9/site-packages/sktime/utils/data_io.py:63: FutureWarning: This function has moved to datasets/_data_io, this version will be removed in V0.10
  warn(
/dati/luca/Uni-Luca/Tesi/progetto/venv/lib/python3.9/site-packages/sktime/utils/data_io.py:63: FutureWarning: This function has moved to datasets/_data_io, this version will be removed in V0.10
  warn(


In [24]:
from keras_tuner import RandomSearch

working_dir = os.path.join("models", DATASET_NAME)
if not os.path.exists(working_dir):
    os.makedirs(working_dir)

tuner = RandomSearch(
    build_ESN,
    # build_ESN
    # build_IRESN, build_IRESNvsr, build_IRESNgsr, build_IRESN_norm_gsr,
    # build_IIRESN, build_IIRESNvsr, build_IIRESNvsr, build_IIRESN_norm_gsr
    objective='val_accuracy',
    max_trials=MAX_TRIALS,
    seed=42,
    directory=working_dir,
    project_name='build_IIRESN',  # change this based on the build function used
    overwrite=OVERWRITE,
    executions_per_trial=TRIALS,
)

tuner.search(x_train, y_train, epochs=MAX_EPOCHS, validation_data=(x_val, y_val),
             callbacks=[
                 keras.callbacks.EarlyStopping(monitor='val_loss', patience=PATIENCE),
             ])

best_model_hp = tuner.get_best_hyperparameters()[0]
tf.random.set_seed(42)

for i in range(BENCHMARKS_TRIALS):
    test_model = tuner.hypermodel.build(best_model_hp)
    history = test_model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=MAX_EPOCHS,
                             callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=PATIENCE,
                                                                      restore_best_weights=True)])
    test_loss, accuracy = test_model.evaluate(x_test, y_test)

    print("     Train accuracy:", history.history['accuracy'][-1])
    print("Validation accuracy:", history.history['val_accuracy'][-1])
    print("      Test accuracy:", accuracy)


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
connectivity      |0.64303           |?                 
spectral radius   |0.067442          |?                 
input scaling     |0.61872           |?                 
bias scaling      |0.41509           |?                 
leaky             |0.88939           |?                 
learning rate     |0.0002555         |?                 

Inputs (470, 3)
Inputs (470, 3)


AttributeError: Exception encountered when calling layer "reservoir" (type Reservoir).

in user code:

    File "/dati/luca/Uni-Luca/Tesi/IRESNs-tensorflow/IRESNs_tensorflow/layer.py", line 180, in call  *
        print("states", states[0].numpy())

    AttributeError: 'Tensor' object has no attribute 'numpy'


Call arguments received:
  • inputs=tf.Tensor(shape=(470, 3), dtype=float32)
  • states=('tf.Tensor(shape=(470, 100), dtype=float32)',)

# Build some custom Models

In [20]:

"""
Experimental model with kernel equals to IRESN and recurrent kernel equals to ESN.
"""

from IRESNs_tensorflow.models import ESNInterface
from IRESNs_tensorflow import layer
from IRESNs_tensorflow import initializers


class CustomESN(ESNInterface):
    def __init__(self,
                 units: int,
                 sub_reservoirs: int,
                 output_units: int,
                 reservoir_activation=tf.keras.activations.tanh,
                 readout_activation=tf.nn.tanh,
                 spectral_radius: float = 0.9,
                 connectivity: float = 1.,
                 input_scaling: float = 1.,
                 bias_scaling=None,
                 leaky=0.1,
                 **kwargs
                 ):
        super().__init__(**kwargs)

        kernel_init = initializers.SplitKernel(sub_reservoirs, input_scaling)

        if connectivity == 1.0:
            recurrent_kernel_init = initializers.FullConnected(spectral_radius)
        else:
            recurrent_kernel_init = initializers.RecurrentKernel(connectivity, spectral_radius)

        self.use_bias = bias_scaling is not None
        if self.use_bias:
            bias_init = tf.keras.initializers.RandomUniform(minval=-bias_scaling, maxval=bias_scaling)
        else:
            bias_init = None

        self.reservoir = keras.Sequential([
            keras.layers.Masking(),
            layer.ESN(units, leaky,
                      activation=reservoir_activation,
                      use_bias=self.use_bias,
                      kernel_initializer=kernel_init,
                      recurrent_initializer=recurrent_kernel_init,
                      bias_initializer=bias_init
                      ),
        ])
        self.readout = keras.Sequential([
            keras.layers.Dense(output_units, activation=readout_activation, name="readout")
        ])


def build_CustomESN(tuner):
    custom_model = CustomESN(units=100,
                             sub_reservoirs=RESERVOIRS,
                             connectivity=tuner.Float('connectivity', min_value=0.01, max_value=1.),
                             spectral_radius=tuner.Float('spectral radius', min_value=MINVAL, max_value=MAXVAL),
                             input_scaling=tuner.Float('input scaling', min_value=MINVAL, max_value=MAXVAL),
                             bias_scaling=tuner.Float('bias scaling', min_value=MINVAL, max_value=MAXVAL),
                             leaky=tuner.Float('leaky', min_value=0.01, max_value=1.0),
                             output_units=OUTPUT_UNITS,
                             readout_activation=READOUT_ACTIVATION
                             )

    alpha = tuner.Float('learning rate', min_value=1e-5, max_value=1e-1, sampling='log')
    custom_model.compile(
        optimizer=keras.optimizers.Adam(alpha),
        loss=LOSS_FUNCTION,
        metrics=['accuracy'],
    )
    return custom_model

In [11]:
tuner = RandomSearch(
    build_CustomESN,  # build_ESN build_IRESN, build_IIRESN, build_IIRESNvsr
    objective='val_accuracy',
    max_trials=MAX_TRIALS,
    seed=42,
    directory=working_dir,
    project_name='CustomESN',  # change this for every model
    overwrite=OVERWRITE,
    executions_per_trial=TRIALS,
)

tuner.search(x_train, y_train, epochs=MAX_EPOCHS, validation_data=(x_val, y_val),
             callbacks=[
                 keras.callbacks.EarlyStopping(monitor='val_loss', patience=PATIENCE),
             ])

Trial 5 Complete [00h 00m 00s]
val_accuracy: 0.06382978707551956

Best val_accuracy So Far: 0.2638297975063324
Total elapsed time: 00h 00m 04s
INFO:tensorflow:Oracle triggered exit
